In [1]:
EXP_NAME = 'LUCAS_Self_ViT_trans'

In [2]:
from train_SimCLR_utils import *
import torch
from torch.utils.data import DataLoader
from dataset.dataset_loader import SNDataset,SNDatasetClimate, myNormalize, myToTensor, Augmentations
from torchvision import transforms
import random
import numpy as np
from dataset.utils.utils import TextColors as tc
from plot_utils.plot import plot_train_test_losses
from datetime import date, datetime
import torch.nn.functional as F
import cv2
import json

In [3]:
DATASET = 'LUCAS' # 'LUCAS', 'RaCA'

if DATASET == 'LUCAS':
    from dataset.dataset_loader import SNDataset,SNDatasetClimate, myNormalize, myToTensor, Augmentations
    OC_MAX = 87
if DATASET == 'RaCA':
    from dataset.dataset_loader import SNDataset,SNDatasetClimate, myNormalize, myToTensor, Augmentations
    OC_MAX = 4115

In [4]:
# create a folder called 'results' in the current directory if it doesn't exist
if not os.path.exists('results_simclr_lucas'):
    os.mkdir('results_simclr_lucas')

In [5]:
# Format the date and time
now = datetime.now()
start_string = now.strftime("%Y-%m-%d %H:%M:%S")
print("Current Date and Time:", start_string)

Current Date and Time: 2024-02-26 13:18:06


In [6]:
import os
os.getcwd()

NAFISEH = "Nafiseh"
MOIEN = "Moien"

if "d:" in os.getcwd():
    USER = MOIEN
elif "c:" in os.getcwd():
    USER = NAFISEH
else:
    raise Exception("Unknown user")

USER

'Nafiseh'

In [7]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
# CONFIG
NUM_WORKERS = 6 if USER == NAFISEH else 2
TRAIN_BATCH_SIZE = 32 if USER == NAFISEH else 4
TEST_BATCH_SIZE = 32 if USER == NAFISEH else 4
LEARNING_RATE = 1e-4 if USER == NAFISEH else 1e-4
NUM_EPOCHS = 30 if USER == NAFISEH else 2
LR_SCHEDULER = "step" # step, plateau or None

USE_SRTM = True

USE_SPATIAL_ATTENTION = False if USER == NAFISEH else False
CNN_ARCHITECTURE = 'ViT' # vgg16 or resnet101 or "ViT"
RNN_ARCHITECTURE = 'Transformer' # LSTM, GRU, RNN, Transformer
REG_VERSION = 1 if USER == NAFISEH else 2
USE_LSTM_BRANCH = True

In [9]:

if USE_SRTM:
    mynorm = myNormalize(img_bands_min_max =[[(0,7),(0,1)], [(7,12),(-1,1)], [(12), (-4,2963)], [(13), (0, 90)]], oc_min = 0, oc_max = OC_MAX)
else:
    mynorm = myNormalize(img_bands_min_max =[[(0,7),(0,1)], [(7,12),(-1,1)]], oc_min = 0, oc_max = OC_MAX)
    
my_to_tensor = myToTensor()
my_augmentation = Augmentations()
train_transform = transforms.Compose([mynorm, my_to_tensor,my_augmentation])
test_transform = transforms.Compose([mynorm, my_to_tensor])

For LUCAS Dataset, we had two different random samples with the same numbering system. In the following we changed the names of one of them. Then we combine the two.

In [10]:
# import os

# # Specify the directory containing the images
# source_directory = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images_simclr_LUCAS"

# # Specify the destination directory for the renamed files
# destination_directory = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Renamed_l8_images_simclr_LUCAS"

# # Create the destination directory if it doesn't exist
# if not os.path.exists(destination_directory):
#     os.makedirs(destination_directory)

# # Loop through all files in the source directory
# for filename in os.listdir(source_directory):
#     # Split the filename on the underscore
#     parts = filename.split("_")
#     if parts[0].startswith("100"):
#         # Replace the first character '1' with '5' and ensure the rest of the number remains unchanged
#         new_number = '5' + parts[0][1:]
#         # Construct the new filename
#         new_filename = f"{new_number}_{parts[1]}"
        
#         # Construct the full old file path and new file path
#         old_file = os.path.join(source_directory, filename)
#         new_file = os.path.join(destination_directory, new_filename)
        
#         # Rename and move the file by changing its path
#         os.rename(old_file, new_file)
#         print(f"Moved and renamed '{old_file}' to '{new_file}'")



In [11]:
# import pandas as pd

# # Specify the source directory containing the CSV files
# source_directory = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR_large\\filled"
# # Specify the destination directory for the updated CSV files
# destination_directory = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR_large\\Renamed_filled"

# # Create the destination directory if it doesn't exist
# if not os.path.exists(destination_directory):
#     os.makedirs(destination_directory)

# # Loop through all files in the source directory
# for filename in os.listdir(source_directory):
#     # Check if the file is a CSV file
#     if filename.endswith(".csv"):
#         # Construct the full file path
#         file_path = os.path.join(source_directory, filename)
        
#         # Read the CSV file into a DataFrame
#         df = pd.read_csv(file_path)
        
#         # Check if 'PointID' column exists in the DataFrame
#         if 'Point_ID' in df.columns:
#             # Update 'PointID' values from '100XXXXX' to '500XXXXX'
#             df['Point_ID'] = df['Point_ID'].apply(lambda x: '500' + str(x)[3:] if str(x).startswith('100') else x)
            
#             # Construct the full path for the updated file
#             updated_file_path = os.path.join(destination_directory, filename)
            
#             # Write the updated DataFrame to a new CSV file in the destination directory
#             df.to_csv(updated_file_path, index=False)
#             print(f"Updated file saved to '{updated_file_path}'")


The combination part is as following

In [12]:
# # Define the paths to the two folders
# folder1 = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\All\\filled'
# folder2 = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR_large\\Renamed_filled'

# # Define the path to the destination folder where you want to save the appended files
# destination_folder = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\LUCAS_SimCLR"

# # Create the destination folder if it doesn't exist
# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)

# # Iterate over the CSV files in the first folder
# for filename in os.listdir(folder1):
#     if filename.endswith(".csv"):
#         file1_path = os.path.join(folder1, filename)
#         file2_path = os.path.join(folder2, filename)
        
#         # Check if the counterpart exists in the second folder
#         if os.path.exists(file2_path):
#             # Read both CSV files
#             df1 = pd.read_csv(file1_path)
#             df2 = pd.read_csv(file2_path)
            
#             # Append df2 to df1
#             appended_df = df1.append(df2, ignore_index=True)
            
#             # Define the path for the appended CSV in the destination folder
#             destination_file_path = os.path.join(destination_folder, filename)
            
#             # Save the appended DataFrame to the destination folder
#             appended_df.to_csv(destination_file_path, index=False)
#             print(f"Appended file saved as '{destination_file_path}'")


Append the samples id: 

In [13]:
# import pandas as pd

# df1 = pd.read_csv("C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\simclr\\simclr.csv")
# df2 = pd.read_csv("C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\simclr\\simclr_LUCAS_large.csv")

# df2['Point_ID'] = df2['Point_ID'].apply(lambda x: '500' + str(x)[3:] if str(x).startswith('100') else x)

# # Append df2 to df1
# appended_df = df1.append(df2, ignore_index=True)

# # Define the path for the output file where you want to save the appended result
# output_file_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\simclr\\LUCAS_SimCLR_1.csv'

# # Save the appended DataFrame to a new CSV file
# appended_df.to_csv(output_file_path, index=False)

# print(f"Appended data saved to '{output_file_path}'")


Check wether there is corrupted image file:

In [14]:
# from skimage import io
# import os

# folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images\\train'

# # Iterate over files in the folder
# for filename in os.listdir(folder_path):
#     # Construct the full path to each image file
#     file_path = os.path.join(folder_path, filename)

#     try:
#         # Load the image
#         l8_img = io.imread(file_path)
#         # Process the loaded image or perform any other operations
#     except Exception as e:
#         print(f"Error loading image {file_path}: {e}")
#         # Handle the error or skip the file if necessary

Check any mismatch between ids of images and climate data

In [15]:
# import os
# import pandas as pd

# # Path to the folder containing the files
# folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images'

# # Path to the CSV file
# csv_file_path = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR_Cleaned\\_soil.csv"

# # Read the CSV file into a DataFrame
# df = pd.read_csv(csv_file_path)

# # Extract the Point_ID column (ensure the column name matches exactly)
# point_ids = df['Point_ID'].unique()

# # List to hold the first part of filenames
# filename_parts = []

# # Loop through all files in the directory
# for filename in os.listdir(folder_path):
#     if "_" in filename:  # Check if filename contains an underscore
#         part = filename.split("_")[0]  # Split the filename and take the first part
#         filename_parts.append(part)

# # Convert filename_parts to a set for faster lookup
# filename_parts_set = set(filename_parts)

# # Check each Point_ID against the filename parts
# for point_id in point_ids:
#     if str(point_id) not in filename_parts_set:
#         print((point_id))


Omit any mismatch data

In [16]:
# import os
# import pandas as pd

# # Directories for your files
# image_dir = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images'
# csv_dir = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR\\"
# clean_csv_dir = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR_Cleaned\\"

# # Ensure the clean CSV directory exists
# os.makedirs(clean_csv_dir, exist_ok=True)

# # List all image filenames and extract Point_IDs
# image_files = os.listdir(image_dir)
# image_point_ids = set(int(f.split('_')[0]) for f in image_files if f.endswith('.tif'))

# # Process each CSV file individually
# for csv_file in os.listdir(csv_dir):
#     if csv_file.endswith('.csv'):
#         # Load the current CSV file
#         csv_path = os.path.join(csv_dir, csv_file)
#         df = pd.read_csv(csv_path)

#         # # Check for missing values
#         # if df.isnull().values.any():
#         #     # Report the presence of missing values
#         #     print(f"Missing values found in {csv_file}:")
#         #     print(df.isnull().sum())

#             # Optional: Handle missing values here (e.g., drop, fill, etc.)
#             # df = df.dropna()  # Example: Remove rows with missing values
#             # df.fillna(0, inplace=True)  # Example: Fill missing values with 0

#         # Filter out rows where Point_ID is not in image_point_ids
#         filtered_df = df[df['Point_ID'].isin(image_point_ids)]

#         # Save the filtered DataFrame to a new file in the clean_csv_dir
#         clean_csv_path = os.path.join(clean_csv_dir, csv_file)
#         filtered_df.to_csv(clean_csv_path, index=False)

#         print(f"Filtered {csv_file} saved to {clean_csv_path}")


In [17]:
if USER == MOIEN:
    train_l8_folder_path = 'D:\python\SoilNet\dataset\l8_images\\train\\'
    test_l8_folder_path = 'D:\python\SoilNet\dataset\l8_images\\test\\'
    val_l8_folder_path = 'D:\python\SoilNet\dataset\l8_images\\val\\'
    testval_l8_folder_path = 'D:\python\SoilNet\dataset\l8_images\\val\\'
    lucas_csv_path = 'D:\python\SoilNet\dataset\LUCAS_2015_all.csv'
    climate_csv_folder_path = "D:\\python\\SoilNet\\dataset\\Climate\\All\\filled\\"


elif USER == NAFISEH:

    if DATASET == 'LUCAS':
     train_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images\\train'
     test_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images\\test'
     val_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images\\val'
     testval_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images\\val'
     # lucas_csv_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-3\\SoilNet\\dataset\\LUCAS_2015_all.csv'
    #  lucas_csv_path = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\simclr\\simclr.csv" # The LUCAS will be replaced by SimCLR file
     lucas_csv_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\simclr\\LUCAS_SimCLR_1.csv'
     # climate_csv_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\All\\filled'
     climate_csv_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR_Cleaned'


    # if DATASET == 'RaCA':
    #  train_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images_simclr_RaCA\\train'
    #  test_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images_simclr_RaCA\\test'
    #  val_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images_simclr_RaCA\\val'
    #  testval_l8_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\l8_images_simclr_RaCA\\test_val'
    #  # lucas_csv_path = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\RapidCarbon_OC.csv"
    #  lucas_csv_path = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\simclr\\simclr_RaCA.csv"
    #  climate_csv_folder_path = 'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\RaCA_SimCLR\\filled'

The 'OC' column should always have a value or the dataloader would face a problem.

In [18]:
# import pandas as pd
# import os

# # Directory containing the CSV files
# csv_dir = "C:\\Users\\nkakhani\\_Multimodal\\SoilNet-7\\SoilNet-PreRelease\\dataset\\Climate\\LUCAS_SimCLR_Cleaned\\"

# # Iterate over each CSV file in the directory
# for csv_file in os.listdir(csv_dir):
#     if csv_file.endswith('.csv'):
#         # Construct the full path to the current CSV file
#         csv_path = os.path.join(csv_dir, csv_file)
        
#         # Read the CSV into a DataFrame
#         df = pd.read_csv(csv_path)
        
#         # Check if 'OC' column exists to avoid KeyError
#         if 'OC' in df.columns:
#             # Replace missing values in 'OC' column with 0
#             df['OC'].fillna(0, inplace=True)
            
#             # Save the modified DataFrame back to the original CSV, overwriting it
#             df.to_csv(csv_path, index=False)
            
#             print(f"Modified '{csv_file}' has been overwritten with the updated 'OC' column values.")
#         else:
#             print(f"'OC' column not found in {csv_file}")


In [19]:
bands = [0,1,2,3,4,5,6,7,8,9,10,11] if not USE_SRTM else [0,1,2,3,4,5,6,7,8,9,10,11,12,13]


################################# IF Not USE_LSTM_BRANCH ###############################
if not USE_LSTM_BRANCH: # NOT USING THE CLIMATE DATA

    train_ds = SNDataset(train_l8_folder_path, lucas_csv_path,l8_bands=bands, transform=train_transform)
    test_ds =  SNDataset(test_l8_folder_path, lucas_csv_path,l8_bands=bands, transform=test_transform)
    val_ds = SNDataset(val_l8_folder_path, lucas_csv_path,l8_bands=bands, transform=test_transform)
    test_ds_w_id =  SNDataset(test_l8_folder_path, lucas_csv_path,l8_bands=bands, transform=test_transform, return_point_id=True)
    testval_ds_w_id =  SNDataset(testval_l8_folder_path, lucas_csv_path,l8_bands=bands, transform=test_transform, return_point_id=True)
    
################################### IF USE_LSTM_BRANCH #################################
else: # USING THE CLIMATE DATA
    train_ds = SNDatasetClimate(train_l8_folder_path,
                                    lucas_csv_path,
                                    climate_csv_folder_path,
                                    l8_bands=bands, transform=train_transform)

    test_ds = SNDatasetClimate(test_l8_folder_path,
                                lucas_csv_path,
                                climate_csv_folder_path,
                                l8_bands=bands, transform=test_transform)
    
    val_ds = SNDatasetClimate(val_l8_folder_path,
                                lucas_csv_path,
                                climate_csv_folder_path,
                                l8_bands=bands, transform=test_transform)
    
    test_ds_w_id = SNDatasetClimate(test_l8_folder_path,
                                lucas_csv_path,
                                climate_csv_folder_path,
                                l8_bands=bands, transform=test_transform, return_point_id=True)

    testval_ds_w_id = SNDatasetClimate(testval_l8_folder_path,
                                lucas_csv_path,
                                climate_csv_folder_path,
                                l8_bands=bands, transform=test_transform, return_point_id=True)
    


In [20]:
SEQ_LEN = test_ds_w_id[0][0][1].shape[0]
SEQ_LEN # max sequence length for Transformer model set up

61

In [21]:
# COUNTING the csv files in the csv folder
CSV_FILES = [f for f in os.listdir(climate_csv_folder_path) if f.endswith('.csv')]
NUM_CLIMATE_FEATURES = len(CSV_FILES)
NUM_CLIMATE_FEATURES

11

In [22]:
from soilnet.soil_net import SoilNet, SoilNetLSTM, SoilNetSimCLR

__name__: submodules.src.transformer.transformer


In [23]:
# train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
# # Get image and label from custom DataLoader
# img_custom, climate,label_custom = next(iter(train_dl))
# print(f"Image shape: {img_custom.shape} -> [batch_size, color_channels, height, width]")
# print(f"Climate shape: {climate.shape} -> [batch_size, climate_channels]")
# print(f"Label shape: {label_custom.shape}")
# img_gpu = img_custom.to(device)
# print(img_gpu.device, img_gpu.shape)
# print(img_gpu.shape)
# y = model(img_gpu)
# y.detach()

In [24]:
cv_results = {"train_loss": [],
                "train_acc_top1": [],
                "train_acc_top5": [],
                "train_acc_mean_pos": [],
                "val_loss": [],
                "val_acc_top1": [],
                "val_acc_top5": [],
                "val_acc_mean_pos": [],
    }

In [25]:
from datetime import date, datetime
# Format the date and time
now = datetime.now()
run_name = now.strftime("D_%Y_%m_%d_T_%H_%M")
print("Current Date and Time:", run_name)
# create a folder called 'results' in the current directory if it doesn't exist
if not os.path.exists('results'):
    os.mkdir('results')

Current Date and Time: D_2024_02_26_T_13_18


In [26]:
# SEEDS = [75] 
SEEDS = [1]  if USER == MOIEN else [1, 4]
# SEEDS = [1] if USER == MOIEN else [1, 4, 69, 75, 79, 128, 474, 786, 2048, 3333]

In [27]:
best_mae = 1000 # just a big number, since our data is normalized between 0 and 1, mae is between 0 and 1 too.
best_seed = SEEDS[0]
for idx, seed in enumerate(SEEDS):
    print(tc.BOLD_BAKGROUNDs.PURPLE, f"CROSS VAL {idx+1}", tc.ENDC)
    
    
    train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
    test_dl = DataLoader(test_ds, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    val_dl = DataLoader(val_ds, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
    
    #model = SoilNetFC(cnn_in_channels=12, regresor_input_from_cnn=1024, hidden_size=128).to(device)
    # architecture = "101+GLAM" if USE_SPATIAL_ATTENTION else "101"
    if USE_LSTM_BRANCH:
        model = SoilNetSimCLR(use_glam=USE_SPATIAL_ATTENTION, cnn_arch= CNN_ARCHITECTURE, reg_version= REG_VERSION,
                           cnn_in_channels=len(bands), regresor_input_from_cnn=128,
                           lstm_n_features= NUM_CLIMATE_FEATURES, lstm_n_layers= 2, lstm_out= 128,
                           hidden_size=128, rnn_arch=RNN_ARCHITECTURE,seq_len=SEQ_LEN).to(device)
    else:
        model = SoilNet(use_glam=USE_SPATIAL_ATTENTION, cnn_arch = CNN_ARCHITECTURE, reg_version= REG_VERSION,
                       cnn_in_channels=len(bands), regresor_input_from_cnn=128, hidden_size=128).to(device)
    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    # Saving the model on the last epoch
    save_model_path = f"results/RUN_{EXP_NAME}_{run_name}_{USER}.pth.tar"
    
    results = train(model, train_dl, test_dl, val_dl,
                    torch.optim.Adam(model.parameters(), lr=LEARNING_RATE),
                    SimCLR(temperature=0.5), epochs=NUM_EPOCHS, lr_scheduler=LR_SCHEDULER,
                    save_model_path= save_model_path,
                    save_model_if_mae_lower_than= best_mae,
                    )

    
    cv_results['train_loss'].append(results['train_loss'])
    cv_results['train_acc_top1'].append(results['train_acc_top1'])
    cv_results['train_acc_top5'].append(results['train_acc_top5'])
    cv_results['train_acc_mean_pos'].append(results['train_acc_mean_pos'])
    cv_results['val_loss'].append(results['val_loss'])
    cv_results['val_acc_top1'].append(results['val_acc_top1'])
    cv_results['val_acc_top5'].append(results['val_acc_top5'])
    cv_results['val_acc_mean_pos'].append(results['val_acc_mean_pos'])
    

    # if results['MAE'][0] < best_mae:
    #     best_mae = results['MAE'][0]
    #     best_seed = seed
    #     print(tc.BOLD_BAKGROUNDs.GREEN, f"MAE improved to {best_mae}", tc.ENDC)
    
        

 CROSS VAL 1 
Adjusting learning rate of group 0 to 1.0000e-04.
 Epoch 1
------------------------------- 


  3%|▎         | 47/1704 [00:22<13:22,  2.07it/s, Train_Loss=4.21] 


KeyboardInterrupt: 

In [ ]:
torch.save(model, f"results/RUN_{EXP_NAME}_{run_name}_{USER}_SelfSupervised.pth")

In [ ]:
train_arr = np.asarray(cv_results['train_acc_mean_pos'])
val_arr = np.asarray(cv_results['val_acc_mean_pos'])
train_arr.shape, val_arr.shape
plot_train_test_losses(train_arr,val_arr, title="Average Self-Rank", x_label="Epochs", y_label="Rank",
                       min_max_bounds= True, tight_x_lim= True,
                       train_legend = "Train", test_legend = "Validation",
                       save_path=f"results/RUN_{run_name}_{USER}.png")

In [ ]:
model = torch.load(f"results/RUN_{EXP_NAME}_{run_name}_{USER}_SelfSupervised.pth")

In [ ]:
train_arr = np.asarray(cv_results['train_acc_top5'])
val_arr = np.asarray(cv_results['val_acc_top5'])
train_arr.shape, val_arr.shape
plot_train_test_losses(train_arr,val_arr, title="Top 5 Prob", x_label="Epochs", y_label="Top 5 Prob",
                       min_max_bounds= True, tight_x_lim= True,
                       train_legend = "Train", test_legend = "Validation",
                       save_path=f"results/RUN_{EXP_NAME}_{run_name}_{USER}.png")

In [ ]:
train_arr = np.asarray(cv_results['train_acc_top1'])
val_arr = np.asarray(cv_results['val_acc_top1'])
train_arr.shape, val_arr.shape
plot_train_test_losses(train_arr,val_arr, title="Top 1 Prob", x_label="Epochs", y_label="Top 1 Prob",
                       min_max_bounds= True, tight_x_lim= True,
                       train_legend = "Train", test_legend = "Validation",
                       save_path=f"results/RUN_{EXP_NAME}_{run_name}_{USER}.png")

# END
We change the rest if the code worked on full dataset.

In [ ]:
import csv

In [ ]:
# Format the date and time
now = datetime.now()
finish_string = now.strftime("%Y-%m-%d %H:%M:%S")
print("Current Date and Time:", finish_string)

In [ ]:
with open(f"results/RUN_{EXP_NAME}_{run_name}_{USER}.json", "w") as fp:
    json.dump(cv_results, fp, indent=4)

In [ ]:
cv_results["MAE"],cv_results['RMSE'],cv_results["R2"]

In [ ]:
cv_results_full = {}
cv_results_full['MAE_MEAN'] = np.mean(cv_results['MAE'])
cv_results_full['RMSE_MEAN'] = np.mean(cv_results['RMSE'])
cv_results_full['R2_MEAN'] = np.mean(cv_results['R2'])
cv_results_full['MAE_MEAN'],cv_results_full['RMSE_MEAN'],cv_results_full['R2_MEAN']

In [ ]:
cv_results_full['USE_LSTM_BRANCH'] = USE_LSTM_BRANCH
cv_results_full['NUM_CLIMATE_FEATURES'] = NUM_CLIMATE_FEATURES if USE_LSTM_BRANCH else None
cv_results_full['CSV_FILES'] = CSV_FILES if USE_LSTM_BRANCH else None
cv_results_full['NUM_WORKERS'] = NUM_WORKERS
cv_results_full['TRAIN_BATCH_SIZE'] = TRAIN_BATCH_SIZE
cv_results_full['TEST_BATCH_SIZE'] = TEST_BATCH_SIZE
cv_results_full['LEARNING_RATE'] = LEARNING_RATE
cv_results_full['NUM_EPOCHS'] = NUM_EPOCHS
cv_results_full['LR_SCHEDULER'] = LR_SCHEDULER
cv_results_full['CNN_ARCHITECTURE'] = CNN_ARCHITECTURE
cv_results_full['REG_VERSION'] = REG_VERSION
cv_results_full['USE_SPATIAL_ATTENTION'] = USE_SPATIAL_ATTENTION
cv_results_full['Best Seed'] = best_seed
cv_results_full['SEEDS'] = SEEDS
cv_results_full['OC_MAX'] = OC_MAX
cv_results_full['USE_SRTM'] = USE_SRTM
cv_results_full['TIME'] = {"start": start_string, "finish": finish_string}
cv_results_full['cv_results'] = cv_results

In [ ]:
with open(f"results/RUN_{EXP_NAME}_{run_name}_{USER}.json", "w") as fp:
    json.dump(cv_results_full, fp, indent=4)

In [ ]:
# READ JSON FILE
# with open("results/my_dict.json", "r") as fp:
#     my_dict = json.load(fp)

# print(my_dict)

# Analysis

## Save Outputs

### Loading the Best Model
Loading the best model from all the coross validation runs.

In [ ]:
load_checkpoint(model=model, optimizer=torch.optim.Adam(model.parameters(), lr=LEARNING_RATE),filename=f"results/RUN_{run_name}_{USER}.pth.tar")
model.eval()
print("Model loaded")

load saved model

In [ ]:
# load_checkpoint(model=model, optimizer=torch.optim.Adam(model.parameters(), lr=LEARNING_RATE),
#                 filename='C:\\Users\\nkakhani\\_Multimodal\\SoilNet-5\\SoilNet\\results\\RUN_D_2023_06_05_T_16_02_Nafiseh.pth.tar') #LSTM 

# load_checkpoint(model=model, optimizer=torch.optim.Adam(model.parameters(), lr=LEARNING_RATE),
#                 filename='C:\\Users\\nkakhani\\_Multimodal\\SoilNet-5\\SoilNet\\results\\RUN_D_2023_06_03_T_09_41_Nafiseh.pth.tar') #GLAM 

# load_checkpoint(model=model, optimizer=torch.optim.Adam(model.parameters(), lr=LEARNING_RATE),
#                 filename='C:\\Users\\nkakhani\\_Multimodal\\SoilNet-5\\SoilNet\\results\\RUN_D_2023_06_02_T_20_57_Nafiseh.pth.tar') #Base

# model.eval()
# print("Model loaded")

In [ ]:
test_dl_w_id = DataLoader(test_ds_w_id, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [ ]:
# test_step_w_id(model=model, data_loader=test_dl_w_id, loss_fn=nn.L1Loss(), verbose=False, csv_file=f"results/RUN_{run_name}_{USER}.csv")
# test_step_w_id(model=model, data_loader=test_dl_w_id, loss_fn=nn.L1Loss(), verbose=False, csv_file=f"results/onlytest_{run_name}_{USER}.csv")

## Visualize Spatial Attention

In [ ]:
if not USE_SPATIAL_ATTENTION or USE_LSTM_BRANCH:
    raise Exception("The Next Part of the Notebook is only available for the model with Spatial Attention and no LSTM Branch")

In [ ]:
BATCH_N = 1

IMG_IN_BATCH = 2

In [ ]:
batch_loader = BatchLoader(test_dl_w_id)

In [ ]:
batch = batch_loader(BATCH_N)
batch[0].shape, batch[1].shape, len(batch[2])

In [ ]:
import sys
import importlib
importlib.reload(sys.modules['plot_utils'])

In [ ]:
from plot_utils.plot import *
from PIL import Image

In [ ]:
# check if results/attention_maps folder exists if not create it
if not os.path.exists("results/attention_maps"):
    os.makedirs("results/attention_maps")

In [ ]:
ABS_ATT = False
ALPHA = 0.4
with torch.no_grad():
    pid = batch[2][IMG_IN_BATCH]
    print("Point Id:", pid)
    out = model(batch[0].to(device))
    print(out.shape)
    print(model.cnn.glam.local_spatial_att.att_map.shape)
    img_rgb = batch[0][0].permute(1,2,0)[:,:,[3,2,1]].detach().cpu().numpy()
    img_rgb = convert2uint8(normalize(img_rgb))
    
    att_map = model.cnn.glam.local_spatial_att.att_map
    att_map = F.interpolate(att_map, size=(64, 64), mode='bicubic', align_corners=True)[IMG_IN_BATCH].detach().cpu().numpy()[0]
    att_map = np.abs(att_map) if ABS_ATT else att_map
    att_map = convert2uint8(normalize(att_map))   
    
    colormap = cv2.applyColorMap(att_map, cv2.COLORMAP_JET)
    
    img_colormaped = cv2.addWeighted(img_rgb, 1 - ALPHA, colormap, ALPHA, 0)

    display_images(img_rgb, img_colormaped, ["Image", "Attention Map"], f"Local Attention Map Visualization | Point ID: {pid}", figsize=(10,5))
    

    
    
    cv2.imwrite(f"results/attention_maps/final/_{run_name}_{USER}_pid{pid}_img_lcl_colormaped.png", cv2.cvtColor(img_colormaped, cv2.COLOR_RGB2BGR))
    
    
    

In [ ]:
GLOB_ATT_CHANNEL = 510 # ranges between 0 and 511
with torch.no_grad():
    pid = batch[2][IMG_IN_BATCH]
    print("Point Id:", pid)
    out = model(batch[0].to(device))
    print(out.shape)
    print(model.cnn.glam.global_spatial_att.att.shape)
    img_rgb = batch[0][0].permute(1,2,0)[:,:,[3,2,1]].detach().cpu().numpy()
    img_rgb = convert2uint8(normalize(img_rgb))
    
    att_map = model.cnn.glam.global_spatial_att.att
    att_map = F.interpolate(att_map, size=(64, 64), mode='bicubic', align_corners=True)[IMG_IN_BATCH].detach().cpu().numpy()[GLOB_ATT_CHANNEL]
    att_map = np.abs(att_map) if ABS_ATT else att_map
    att_map = convert2uint8(normalize(att_map))   
    
    colormap = cv2.applyColorMap(att_map, cv2.COLORMAP_JET)
    
    img_colormaped = cv2.addWeighted(img_rgb, 1 - ALPHA, colormap, ALPHA, 0)

    display_images(img_rgb, img_colormaped, ["Image", "Attention Map"], f"Global{GLOB_ATT_CHANNEL} Attention Map Visualization | Point ID: {pid}") 
                #    figsize=(10,5), savepath=f"results/attention_maps/final/RUN_{run_name}_{USER}_pid{pid}_glb{GLOB_ATT_CHANNEL}.png")
    
    # # Save img_rgband img_colormaped as jpg files
    # cv2.imwrite(f"results/attention_maps/RUN_{run_name}_{USER}_pid{pid}_img_glb{GLOB_ATT_CHANNEL}_rgb.png", cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR))
    # cv2.imwrite(f"results/attention_maps/RUN_{run_name}_{USER}_pid{pid}_img_glb{GLOB_ATT_CHANNEL}_colormaped.png", cv2.cvtColor(img_colormaped, cv2.COLOR_RGB2BGR))
    
    
    